In [1]:
from skimage import transform, filters, exposure
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Conv2D

Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import os
import glob
import pandas as pd
import math
from PIL import Image
import numpy as np
from scipy.misc import imread, imsave, imresize
from natsort import natsorted
from scipy.misc import imread
import random
random.seed( 42 ) 

nb_classes=62
# Path of data files
path = "/home/ivan/python/julia/data"
 
# Input image dimensions
img_rows, img_cols = 20, 20
 
 

### Images preprocessing ###

for setType in ["test","train"]:
    # We have to make sure files are sorted according to labels, even if they don't have trailing zeros
    files = natsorted(glob.glob(path + "/"+setType+"/*"))
    """
    if setType == 'test':
        X_test = np.array([np.array(Image.open(fname)) for fname in files])
    else:
        X_train = np.array([np.array(Image.open(fname)) for fname in files])
    """
    if setType == 'test':
        X_test = np.array([np.array(imread(fname, flatten=True)) for fname in files])
    else:
        X_train = np.array([np.array(imread(fname, flatten=True)) for fname in files])
X_test=X_test[:,:,:,np.newaxis]
X_train=X_train[:,:,:,np.newaxis]
X_train /= 255
X_test /= 255
X_test.shape
    

(6220, 20, 20, 1)

In [3]:
def label2int(ch):
    asciiVal = ord(ch)
    if(asciiVal<=57): #0-9
        asciiVal-=48
    elif(asciiVal<=90): #A-Z
        asciiVal-=55
    else: #a-z
        asciiVal-=61
    return asciiVal
    
def int2label(i):
    if(i<=9): #0-9
        i+=48
    elif(i<=35): #A-Z
        i+=55
    else: #a-z
        i+=61
    return chr(i)
# Load labels
y_train = pd.read_csv("/home/ivan/python/julia/trainLabels.csv").values[:,1] #Keep only label
 
# Convert labels to one-hot vectors
Y_train = np.zeros((y_train.shape[0], len(np.unique(y_train))))
 
for i in range(y_train.shape[0]):
    Y_train[i][label2int(y_train[i])] = 1 # One-hot
Y_train_all = Y_train.copy()
X_train_all = X_train.copy()

In [4]:
X_train, X_val, Y_train, Y_val = \
        train_test_split(X_train_all, Y_train_all, test_size=0.2, stratify=np.argmax(Y_train_all, axis=1))

In [7]:
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout, concatenate, Conv2DTranspose
inputs = Input((img_rows, img_cols, 1))
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv3), conv2], axis=3)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv1], axis=3)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv7)

"""
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)

up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)

up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)

up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)

up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
"""

flat = Flatten()(conv7)
dense1 = Dense(2048, activation="relu")(flat)
drp1 = Dropout(0.5)(dense1)
dense2 = Dense(2048, activation="relu")(drp1)
drp2 = Dropout(0.5)(dense2)
dense3 = Dense(nb_classes, activation="softmax")(drp1)

model = Model(input=inputs, output=dense3)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:56: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [8]:
### LEARNING ###
from keras.optimizers import Adam, Adadelta
# First, use AdaDelta for some epochs because AdaMax gets stuck
model.compile(loss='categorical_crossentropy', 
              optimizer=Adadelta(),  
              metrics=["accuracy"])

In [9]:
# 20 epochs is sufficient
model.fit(X_train, Y_train, batch_size=128,
                    #nb_epoch=20, 
                    epochs=20,
                    validation_data=(X_val, Y_val),
                    verbose=1)

Train on 5026 samples, validate on 1257 samples
Epoch 1/20
5026/5026 [==============================] - 284s - loss: 3.8626 - acc: 0.0609 - val_loss: 3.7924 - val_acc: 0.0573
Epoch 2/20
5026/5026 [==============================] - 281s - loss: 3.8083 - acc: 0.0625 - val_loss: 3.7728 - val_acc: 0.0732
Epoch 3/20
5026/5026 [==============================] - 289s - loss: 3.7868 - acc: 0.0760 - val_loss: 3.7500 - val_acc: 0.0859
Epoch 4/20
5026/5026 [==============================] - 285s - loss: 3.7082 - acc: 0.1110 - val_loss: 3.5653 - val_acc: 0.1583
Epoch 5/20
5026/5026 [==============================] - 278s - loss: 3.3638 - acc: 0.2099 - val_loss: 3.2375 - val_acc: 0.1854
Epoch 6/20
5026/5026 [==============================] - 270s - loss: 2.8135 - acc: 0.3227 - val_loss: 2.9137 - val_acc: 0.2928
Epoch 7/20
5026/5026 [==============================] - 271s - loss: 2.1884 - acc: 0.4443 - val_loss: 2.2914 - val_acc: 0.4105
Epoch 8/20
5026/5026 [==============================] - 266s - 

In [10]:
# Parametrize the image augmentation class
datagen = ImageDataGenerator(
    rotation_range = 20,
    width_shift_range = 0.15,
    height_shift_range = 0.15,
    shear_range = 0.4,
    zoom_range = 0.3,                    
    channel_shift_range = 0.1)

In [11]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adamax',  
              metrics=["accuracy"])

# We want to keep the best model. This callback will store 
# in a file the weights of the model with the highest validation accuracy  
saveBestModel = ModelCheckpoint("best.kerasModelWeights", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True)

# Make the model learn using the image generator
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
                samples_per_epoch=len(X_train),
                epochs=300, 
                validation_data=(X_val, Y_val),
                callbacks=[saveBestModel],
                verbose=1)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, steps_per_epoch=39, callbacks=[<keras.ca..., validation_data=(array([[[..., epochs=300)`
  from ipykernel import kernelapp as app


Epoch 1/300
39/39 [==============================] - 139s - loss: 3.5403 - acc: 0.1589 - val_loss: 2.3656 - val_acc: 0.4543
Epoch 2/300
39/39 [==============================] - 136s - loss: 2.9172 - acc: 0.2775 - val_loss: 1.8235 - val_acc: 0.5442
Epoch 3/300
39/39 [==============================] - 136s - loss: 2.7082 - acc: 0.3139 - val_loss: 1.5916 - val_acc: 0.5982
Epoch 4/300
39/39 [==============================] - 135s - loss: 2.5277 - acc: 0.3534 - val_loss: 1.4928 - val_acc: 0.6070
Epoch 5/300
39/39 [==============================] - 135s - loss: 2.3772 - acc: 0.3970 - val_loss: 1.3920 - val_acc: 0.6476
Epoch 6/300
39/39 [==============================] - 135s - loss: 2.2772 - acc: 0.4089 - val_loss: 1.3065 - val_acc: 0.6659
Epoch 7/300
39/39 [==============================] - 135s - loss: 2.2300 - acc: 0.4173 - val_loss: 1.2616 - val_acc: 0.6667
Epoch 8/300
39/39 [==============================] - 135s - loss: 2.1769 - acc: 0.4273 - val_loss: 1.2187 - val_acc: 0.6826
Epoch 9/

39/39 [==============================] - 135s - loss: 1.2960 - acc: 0.6416 - val_loss: 0.8649 - val_acc: 0.7677
Epoch 32/300
39/39 [==============================] - 136s - loss: 1.2762 - acc: 0.6390 - val_loss: 0.9235 - val_acc: 0.7574
Epoch 33/300
39/39 [==============================] - 138s - loss: 1.2755 - acc: 0.6411 - val_loss: 0.8876 - val_acc: 0.7605
Epoch 34/300
39/39 [==============================] - 134s - loss: 1.2623 - acc: 0.6388 - val_loss: 0.8528 - val_acc: 0.7677
Epoch 35/300
39/39 [==============================] - 134s - loss: 1.2102 - acc: 0.6532 - val_loss: 0.8910 - val_acc: 0.7605
Epoch 36/300
39/39 [==============================] - 135s - loss: 1.2028 - acc: 0.6542 - val_loss: 0.8728 - val_acc: 0.7653
Epoch 37/300
39/39 [==============================] - 135s - loss: 1.2138 - acc: 0.6490 - val_loss: 0.8865 - val_acc: 0.7510
Epoch 38/300
39/39 [==============================] - 136s - loss: 1.1742 - acc: 0.6615 - val_loss: 0.8415 - val_acc: 0.7661
Epoch 39/300


39/39 [==============================] - 136s - loss: 0.8396 - acc: 0.7500 - val_loss: 0.8005 - val_acc: 0.7836
Epoch 64/300
39/39 [==============================] - 136s - loss: 0.8430 - acc: 0.7407 - val_loss: 0.8335 - val_acc: 0.7757
Epoch 65/300
39/39 [==============================] - 136s - loss: 0.8336 - acc: 0.7477 - val_loss: 0.8313 - val_acc: 0.7868
Epoch 66/300
39/39 [==============================] - 136s - loss: 0.8504 - acc: 0.7421 - val_loss: 0.7736 - val_acc: 0.7876
Epoch 67/300
39/39 [==============================] - 136s - loss: 0.7583 - acc: 0.7750 - val_loss: 0.8176 - val_acc: 0.7852
Epoch 68/300
39/39 [==============================] - 136s - loss: 0.7921 - acc: 0.7553 - val_loss: 0.8379 - val_acc: 0.7900
Epoch 69/300
39/39 [==============================] - 136s - loss: 0.7935 - acc: 0.7637 - val_loss: 0.8170 - val_acc: 0.7717
Epoch 70/300
39/39 [==============================] - 136s - loss: 0.7493 - acc: 0.7616 - val_loss: 0.8693 - val_acc: 0.7796
Epoch 71/300


39/39 [==============================] - 136s - loss: 0.6250 - acc: 0.8060 - val_loss: 0.8192 - val_acc: 0.8019
Epoch 98/300
39/39 [==============================] - 136s - loss: 0.5741 - acc: 0.8154 - val_loss: 0.7838 - val_acc: 0.8003
Epoch 99/300
39/39 [==============================] - 136s - loss: 0.6143 - acc: 0.8029 - val_loss: 0.8305 - val_acc: 0.7916
Epoch 100/300
39/39 [==============================] - 136s - loss: 0.6148 - acc: 0.8076 - val_loss: 0.8231 - val_acc: 0.7955
Epoch 101/300
39/39 [==============================] - 136s - loss: 0.5734 - acc: 0.8166 - val_loss: 0.8538 - val_acc: 0.7963
Epoch 102/300
39/39 [==============================] - 136s - loss: 0.5817 - acc: 0.8177 - val_loss: 0.8206 - val_acc: 0.7955
Epoch 103/300
39/39 [==============================] - 136s - loss: 0.5835 - acc: 0.8175 - val_loss: 0.7962 - val_acc: 0.7876
Epoch 104/300
39/39 [==============================] - 136s - loss: 0.5575 - acc: 0.8231 - val_loss: 0.8345 - val_acc: 0.7932
Epoch 10

...

39/39 [==============================] - 136s - loss: 0.4091 - acc: 0.8650 - val_loss: 0.9260 - val_acc: 0.7812
Epoch 165/300
39/39 [==============================] - 136s - loss: 0.4134 - acc: 0.8672 - val_loss: 0.9165 - val_acc: 0.7971
Epoch 166/300
39/39 [==============================] - 136s - loss: 0.3945 - acc: 0.8693 - val_loss: 0.9548 - val_acc: 0.7876
Epoch 167/300
39/39 [==============================] - 136s - loss: 0.3667 - acc: 0.8781 - val_loss: 0.8973 - val_acc: 0.7884
Epoch 168/300
39/39 [==============================] - 136s - loss: 0.3920 - acc: 0.8717 - val_loss: 0.8958 - val_acc: 0.7796
Epoch 169/300
39/39 [==============================] - 136s - loss: 0.3931 - acc: 0.8700 - val_loss: 0.9551 - val_acc: 0.7828
Epoch 170/300
39/39 [==============================] - 136s - loss: 0.3606 - acc: 0.8811 - val_loss: 0.9418 - val_acc: 0.7828
Epoch 171/300
39/39 [==============================] - 136s - loss: 0.3749 - acc: 0.8747 - val_loss: 0.9768 - val_acc: 0.7884
Epoch 

39/39 [==============================] - 136s - loss: 0.3395 - acc: 0.8893 - val_loss: 0.8992 - val_acc: 0.7932
Epoch 199/300
39/39 [==============================] - 136s - loss: 0.3523 - acc: 0.8839 - val_loss: 0.9408 - val_acc: 0.7924
Epoch 200/300
39/39 [==============================] - 137s - loss: 0.3573 - acc: 0.8890 - val_loss: 0.9530 - val_acc: 0.7963
Epoch 201/300
39/39 [==============================] - 138s - loss: 0.3430 - acc: 0.8856 - val_loss: 0.9007 - val_acc: 0.7963
Epoch 202/300
39/39 [==============================] - 136s - loss: 0.3291 - acc: 0.8864 - val_loss: 0.9647 - val_acc: 0.7955
Epoch 203/300
39/39 [==============================] - 136s - loss: 0.3335 - acc: 0.8918 - val_loss: 0.9506 - val_acc: 0.7900
Epoch 204/300
39/39 [==============================] - 136s - loss: 0.3263 - acc: 0.8926 - val_loss: 0.9375 - val_acc: 0.7995
Epoch 205/300
39/39 [==============================] - 135s - loss: 0.3156 - acc: 0.8931 - val_loss: 0.9277 - val_acc: 0.7995
Epoch 

39/39 [==============================] - 136s - loss: 0.3140 - acc: 0.8973 - val_loss: 0.9356 - val_acc: 0.7860
Epoch 233/300
39/39 [==============================] - 136s - loss: 0.3011 - acc: 0.9028 - val_loss: 0.9575 - val_acc: 0.8035
Epoch 234/300
39/39 [==============================] - 136s - loss: 0.3167 - acc: 0.8900 - val_loss: 0.9795 - val_acc: 0.7908
Epoch 235/300
39/39 [==============================] - 136s - loss: 0.3344 - acc: 0.8912 - val_loss: 0.9103 - val_acc: 0.7844
Epoch 236/300
39/39 [==============================] - 136s - loss: 0.2858 - acc: 0.9088 - val_loss: 0.9470 - val_acc: 0.7987
Epoch 237/300
39/39 [==============================] - 137s - loss: 0.2782 - acc: 0.9088 - val_loss: 0.9811 - val_acc: 0.7924
Epoch 238/300
39/39 [==============================] - 136s - loss: 0.2992 - acc: 0.8985 - val_loss: 0.9906 - val_acc: 0.7908
Epoch 239/300
39/39 [==============================] - 136s - loss: 0.3016 - acc: 0.9017 - val_loss: 0.9383 - val_acc: 0.7932
Epoch 

39/39 [==============================] - 144s - loss: 0.2853 - acc: 0.9084 - val_loss: 0.9346 - val_acc: 0.7924
Epoch 267/300
39/39 [==============================] - 135s - loss: 0.2668 - acc: 0.9165 - val_loss: 1.0084 - val_acc: 0.7940
Epoch 268/300
39/39 [==============================] - 136s - loss: 0.2815 - acc: 0.9049 - val_loss: 0.9869 - val_acc: 0.7979
Epoch 269/300
39/39 [==============================] - 135s - loss: 0.2749 - acc: 0.9062 - val_loss: 1.0305 - val_acc: 0.8035
Epoch 270/300
39/39 [==============================] - 136s - loss: 0.2910 - acc: 0.9034 - val_loss: 0.9728 - val_acc: 0.7892
Epoch 271/300
39/39 [==============================] - 142s - loss: 0.2783 - acc: 0.9108 - val_loss: 1.0028 - val_acc: 0.7852
Epoch 272/300
39/39 [==============================] - 139s - loss: 0.2684 - acc: 0.9103 - val_loss: 0.9967 - val_acc: 0.7852
Epoch 273/300
39/39 [==============================] - 138s - loss: 0.2961 - acc: 0.9026 - val_loss: 0.9752 - val_acc: 0.7860
Epoch 

39/39 [==============================] - 147s - loss: 0.2464 - acc: 0.9196 - val_loss: 1.0130 - val_acc: 0.7916


In [13]:
# Predict the class (give the index in the one-hot vector of the most probable class)
Y_test_pred = model.predict_classes(X_test, verbose=1)

# Translate integers to character labels
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(Y_test_pred)

# Save the predicitions in Kaggle format
np.savetxt(path+"/CNN_pred_unet.csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')



AttributeError: 'Model' object has no attribute 'predict_classes'

In [14]:
y_pred=model.predict(X_test,verbose=1)

In [16]:
y_pred1 = np.argmax(y_pred,axis=1)

In [17]:
y_pred1

array([17, 14, 18, ..., 27, 23, 22])

In [18]:
vInt2label = np.vectorize(int2label)
Y_test_pred = vInt2label(y_pred1)

# Save the predicitions in Kaggle format
np.savetxt(path+"/CNN_pred_unet.csv", np.c_[range(6284,len(Y_test_pred)+6284),Y_test_pred], delimiter=',', header = 'ID,Class', comments = '', fmt='%s')
